## Objectives:
* Step 1: prepare the training and evaluation data set
* Step 2: training with random forest

In [1]:
import pandas as pd
import numpy as np
import datetime
import calendar
from sklearn import preprocessing
from sklearn import feature_extraction
import itertools
from collections import OrderedDict

# To use part or all the train set
training = False

#### Prepare the tables

In [2]:
#Load all Files (hey must be in input directory in a brother directory of the notebook)
data_load = {
    'item_categories': pd.read_csv('../input/item_categories.csv'), 
    'items': pd.read_csv('../input/items.csv'), 
    'sales_train': pd.read_csv('../input/sales_train_v2.csv'),
    'sample_submission': pd.read_csv('../input/sample_submission.csv'),
    'shops': pd.read_csv('../input/shops.csv'),
    'test': pd.read_csv('../input/test.csv')
}

In [3]:
Data = {}

# Sales data 
data_load['sales_train']['date'] = pd.to_datetime(data_load['sales_train']['date'], format = "%d.%m.%Y")

#Data['sales']['day'] = transactions['date'].dt.day
data_load['sales_train']['month'] = data_load['sales_train']['date'].dt.month
data_load['sales_train']['year'] = data_load['sales_train']['date'].dt.year

In [4]:
data_load['sales_train'][(data_load['sales_train'].shop_id == 5) & (data_load['sales_train'].item_id == 485)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year
669316,2013-07-23,6,5,485,300.0,1.0,7,2013
669317,2013-07-25,6,5,485,300.0,1.0,7,2013
827143,2013-08-05,7,5,485,300.0,1.0,8,2013
877001,2013-09-07,8,5,485,300.0,1.0,9,2013
1401436,2014-02-28,13,5,485,300.0,1.0,2,2014


In [5]:
data_load['sales_train'].describe(include = 'all')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year
count,2935849,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
unique,1034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2013-12-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,9434,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2015-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,6.247717e+00,2.013777e+03
std,NaN,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,3.536219e+00,7.684790e-01
min,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,1.000000e+00,2.013000e+03
25%,NaN,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,3.000000e+00,2.013000e+03


### Split the training/evaluation set, with similar pattern as the test set:
* All shops in both sets
* evaluation set on the last month
* unknown items in the evaluation set

In [5]:
import random

# Calculate the total of items sold per month, item, shop

Data['train'] = data_load['sales_train'].groupby(['date_block_num', 'shop_id', 'item_id'], as_index = False).item_cnt_day.agg({
    'item_cnt_month': np.sum
})

# We calculate the average per line, not weighted by the number of sales to simplify (and avoid division per 0)

tmp = data_load['sales_train'].groupby(['date_block_num', 'shop_id', 'item_id'], as_index= False).item_price.agg({
    'item_mean_price_shop_month': np.mean
})

Data['train'] = Data['train'].merge(tmp,
                    how = 'left',
                    on = ['date_block_num', 'shop_id', 'item_id'])

if training: 
    
    # Split on date
    
    condition = Data['train']['date_block_num']==33
    Data['evaluation'] = Data['train'][condition]
    Data['train'] = Data['train'][~condition]
    
    print("sizes:" ,Data['evaluation'].shape, Data['train'].shape)

    # The following part was disable, because the evaluation set already presents a larger proportion of unknown items than 
    # the test set
    
    # Select 7.1% of the items to remove from the data set (363 on 5100 )
    # This may be too high, as some items could be appearing only in this last month: TO CHECK > in fact the proportion 
    # of missing item is already too large

    #list_item_ids = list(Data['train'].item_id.unique())
    #n = int(363/5100*21807)

    # Remove the items from training set

    #removed_item_ids = random.sample(list_item_ids, n)
    #print("Number of items removed from train set:", len(removed_item_ids))

    #condition = Data['train'].item_id.isin(removed_item_ids)
    #Data['train'] = Data['train'][~condition]

    #print(Data['evaluation'].shape, Data['train'].shape)

In [7]:
 Data['train'][( Data['train'].shop_id == 5) & ( Data['train'].item_id == 485)]

,date_block_num,shop_id,item_id,item_cnt_month,item_mean_price_shop_month
353061,6,5,485,2.0,300.0
411402,7,5,485,1.0,300.0
468936,8,5,485,1.0,300.0
742695,13,5,485,1.0,300.0


### Winsorization - on training set only to evaluate the effect on prediction

In [6]:
price_limit = Data['train'].item_mean_price_shop_month.quantile([0.0, 0.999])[0.999]
prices = Data['train'].item_mean_price_shop_month
Data['train'].loc[(prices > price_limit), 'item_mean_price_shop_month'] = price_limit

# clipping must be done after the stats are calculated
#Data['train'].loc[:,'item_cnt_month'] = Data['train'].item_cnt_month.clip(0,20)

#### Add the missing rows

In [7]:
if training:
    set_list = ['train', 'evaluation']
else:
    set_list = ['train']
    
months = data_load['sales_train'].groupby(['month', 'year'], as_index = False).date_block_num.first()

for set_name in set_list:

    # Add the missing rows date_block_num,shop_id, item_id with item_cnt_month = 0
    
    print('Adding the missing rows for', set_name, 'set...')
    unique_date_block_num = sorted(Data[set_name].date_block_num.unique())
    unique_shop_id = sorted(Data[set_name].shop_id.unique())
    unique_item_id = sorted(Data[set_name].item_id.unique())

    d = {
        'date_block_num': unique_date_block_num,
        'item_id': unique_item_id,
        'shop_id': unique_shop_id
    }

    tmp = list(itertools.product(*[unique_date_block_num, unique_item_id, unique_shop_id]))
    od = OrderedDict(sorted(d.items()))

    df = pd.DataFrame(tmp,columns=od.keys())
    Data[set_name] = df.merge(Data[set_name],
                             how= 'left',
                             on = ['shop_id', 'item_id', 'date_block_num'])

    # Add the month and year
    
    Data[set_name] = Data[set_name].merge(months,
                   how = 'left',
                   on = ['date_block_num'])
    
    # For new rows missing values:
    
    Data[set_name].item_cnt_month = Data[set_name].item_cnt_month.fillna(0)

Adding the missing rows for train set...


In [8]:
Data['train'].describe()

,date_block_num,item_id,shop_id,item_cnt_month,item_mean_price_shop_month,month,year
count,4.448628e+07,4.448628e+07,4.448628e+07,4.448628e+07,1.609124e+06,4.448628e+07,4.448628e+07
mean,1.650000e+01,1.109870e+04,2.950000e+01,8.200744e-02,7.867442e+02,6.205882e+00,2.013941e+03
std,9.810709e+00,6.396913e+03,1.731810e+01,1.698690e+00,1.466117e+03,3.323529e+00,8.022460e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,9.000000e-02,1.000000e+00,2.013000e+03
25%,8.000000e+00,5.551000e+03,1.475000e+01,0.000000e+00,1.990000e+02,3.000000e+00,2.013000e+03
50%,1.650000e+01,1.110500e+04,2.950000e+01,0.000000e+00,3.990000e+02,6.000000e+00,2.014000e+03
75%,2.500000e+01,1.664800e+04,4.425000e+01,0.000000e+00,8.985000e+02,9.000000e+00,2.015000e+03
max,3.300000e+01,2.216900e+04,5.900000e+01,2.253000e+03,2.269199e+04,1.200000e+01,2.015000e+03


#### Add the other features

In [9]:
# 'train' must be first in those lists
if training:
    set_list = ['train','evaluation']
else:
    set_list = ['train', 'test']

if ~training:
    # Prepare the test set
    print("Preparing the test set")

    Data['test'] = data_load['test'].copy()
    Data['test']['month'] = 11
    Data['test']['year'] = 2015
    Data['test']['date_block_num'] = 34

    cols = ['ID', 'date_block_num', 'item_id', 'shop_id','month', 'year']
    Data['test'] = Data['test'][cols]

#print(Data['test'].columns.tolist())

### Add the item_category

print("Adding item category stats")

for set_name in set_list:
    Data[set_name] = Data[set_name].merge(data_load['items'],
            how = 'left',
            on = 'item_id',
            ).drop(['item_name'], axis = 1)

# Stats over the items and the months
tmp = Data['train'].groupby(['shop_id', 'item_category_id'], as_index = False).item_cnt_month.agg({
    'category_cnt_month_mean': np.mean,   
    'category_cnt_month_max': np.max
})

for set_name in set_list:
    Data[set_name] = Data[set_name].merge(tmp,
            how = 'left',
            on = ['item_category_id', 'shop_id']
            )
    
# The category without sales for this restaurant will remain with a NaN (removed when creating the X/y)

### Add the lagged information on item count
print("Adding item count lagged values")

Data['train']['item_cnt_month_minus_12'] = 0
Data['train']['item_cnt_month_minus_3'] = 0
Data['train']['item_cnt_month_minus_1'] = 0 

for it_month in range(12, 33,1):
    condition = Data['train'].date_block_num == it_month
    Data['train'].loc[condition, 'item_cnt_month_minus_12'] = Data['train'][(Data['train'].date_block_num == it_month-12)]\
        .as_matrix(columns=['item_cnt_month'])    
    Data['train'].loc[condition, 'item_cnt_month_minus_3'] = Data['train'][(Data['train'].date_block_num == it_month-3)]\
        .as_matrix(columns=['item_cnt_month'])
    Data['train'].loc[condition, 'item_cnt_month_minus_1'] = Data['train'][(Data['train'].date_block_num == it_month-1)]\
        .as_matrix(columns=['item_cnt_month'])

if training:
    for lag in [1,3,12]:
        print("Evaluation set - Calculating lag:",lag)
        tmp = Data['train'][Data['train'].date_block_num == (33-lag)][['item_id', 'shop_id','item_cnt_month']]\
            .rename(columns={'item_cnt_month':'item_cnt_month_minus_'+str(lag) })
        Data['evaluation'] = Data['evaluation'].merge(tmp,
                how = 'left',
                on = ['item_id', 'shop_id']
                )    
else:     
    for lag in [1,3,12]:
        print("Test set - Calculating lag:",lag)
        tmp = Data['train'][Data['train'].date_block_num == (34-lag)][['item_id', 'shop_id','item_cnt_month']]\
            .rename(columns={'item_cnt_month':'item_cnt_month_minus_'+str(lag) })
        Data['test'] = Data['test'].merge(tmp,
                how = 'left',
                on = ['item_id', 'shop_id']
                )

### Add the stats on item_cnt_month for every couple store,item
# Only known values are for the train set

print("Adding item count stats")

# Stats over the months
tmp = Data['train'].groupby(['shop_id', 'item_id'], as_index = False).item_cnt_month.agg({
    'item_cnt_month_mean': np.mean,
    'item_cnt_month_max': np.max
})
# Warning: if the couple item, shop is not present in the train set for any month, the following will generate a NaN:
# * for merge with the train set: this will produce a 0 (all couples exist) > we remove those later as they are all the same
# and cannot be used to learn anything
# * for merge with evaluation/test set: this will produce a NaN > those lines will be replaced by stats
# > this is wrong, the system cannot learn, we have to replace those lines in the eval/test after prediction

for set_name in set_list:
    Data[set_name] = Data[set_name].merge(tmp,
            how = 'left',
            on = ['item_id', 'shop_id']
            )
    
# Add the average price per item and shop, over the train data set (average price will be missing for some items)

print("Adding shop item price stats")

# Average over the months of the train set per item and shop
# The mean is not weighted by the sales

tmp = Data['train'].groupby(['item_id', 'shop_id'], as_index = False).item_mean_price_shop_month.agg({
    'item_shop_mean_price': np.mean
})

for set_name in set_list:
    Data[set_name] = Data[set_name].merge(tmp,
            how = 'left',
            on = ['item_id', 'shop_id']
            )

# We remove the couple (item, shop) without any sale from the training set
condition = pd.isnull(Data['train'].item_shop_mean_price)
Data['train'] = Data['train'][~condition]
    
    #missing_shop_item_indices = pd.isnull(Data[set_name]['item_shop_mean_price'])
    #Data[set_name].loc[missing_shop_item_indices, 'item_shop_mean_price'] = -999

print("Adding overall item price stats")    
    
# Add the average price per item over the train data set (average price will be missing for some items)
# The mean is not weighted by the sales

tmp = Data['train'].groupby(['item_id'], as_index = False).item_mean_price_shop_month.agg({
    'item_overall_mean_price': np.mean
})

for set_name in set_list:
    Data[set_name] = Data[set_name].merge(tmp,
            how = 'left',
            on = ['item_id']
            )
    # The item with no sale in the training set will be addressed after prediction
    # missing_item_indices = pd.isnull(Data[set_name]['item_overall_mean_price'])
    # Data[set_name].loc[missing_item_indices, 'item_overall_mean_price'] = -999

# Remove the item_id TO PUT BACK

#for set_name in set_list:
#    Data[set_name] = Data[set_name].drop(['item_id', 'item_category_id'], axis = 1)
    
Data['train'] = Data['train'].drop(['item_mean_price_shop_month'], axis = 1)
if training:
    Data['evaluation'] = Data['evaluation'].drop(['item_mean_price_shop_month'], axis = 1)

print("Done!")
#Data['evaluation'].head(10)

Preparing the test set
Adding item category stats
Adding item count lagged values
Test set - Calculating lag: 1
Test set - Calculating lag: 3
Test set - Calculating lag: 12
Adding item count stats
Adding shop item price stats
Adding overall item price stats
Done!


In [10]:
Data['train'].describe()

,date_block_num,item_id,shop_id,item_cnt_month,month,year,item_category_id,category_cnt_month_max,category_cnt_month_mean,item_cnt_month_minus_12,item_cnt_month_minus_3,item_cnt_month_minus_1,item_cnt_month_mean,item_cnt_month_max,item_shop_mean_price,item_overall_mean_price
count,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07,1.442022e+07
mean,1.650000e+01,1.145802e+04,3.143122e+01,2.529925e-01,6.205882e+00,2.013941e+03,4.260401e+01,4.808575e+01,1.536073e-01,1.726465e-01,1.562529e-01,1.473567e-01,2.529925e-01,2.966234e+00,7.898727e+02,7.930833e+02
std,9.810709e+00,6.133325e+03,1.696204e+01,2.976348e+00,3.323530e+00,8.022460e-01,1.521945e+01,9.141636e+01,5.268633e-01,2.220762e+00,2.417708e+00,2.296853e+00,1.897223e+00,1.115443e+01,1.609214e+03,1.606589e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,1.000000e+00,2.013000e+03,0.000000e+00,1.000000e+00,3.824677e-05,-2.200000e+01,-2.200000e+01,-5.000000e+00,-3.529412e-01,0.000000e+00,9.000000e-02,4.895753e+00
25%,8.000000e+00,6.244000e+03,1.800000e+01,0.000000e+00,3.000000e+00,2.013000e+03,3.700000e+01,1.100000e+01,4.317897e-02,0.000000e+00,0.000000e+00,0.000000e+00,2.941176e-02,1.000000e+00,1.990000e+02,2.010241e+02
50%,1.650000e+01,1.161400e+04,3.000000e+01,0.000000e+00,6.000000e+00,2.014000e+03,4.000000e+01,2.100000e+01,7.665782e-02,0.000000e+00,0.000000e+00,0.000000e+00,8.823529e-02,1.000000e+00,3.490000e+02,3.442273e+02
75%,2.500000e+01,1.666200e+04,4.600000e+01,0.000000e+00,9.000000e+00,2.015000e+03,5.500000e+01,5.200000e+01,1.858289e-01,0.000000e+00,0.000000e+00,0.000000e+00,2.058824e-01,2.000000e+00,7.990000e+02,8.121671e+02
max,3.300000e+01,2.216900e+04,5.900000e+01,2.253000e+03,1.200000e+01,2.015000e+03,8.300000e+01,2.253000e+03,9.893137e+01,1.305000e+03,1.305000e+03,1.305000e+03,5.862941e+02,2.253000e+03,2.269199e+04,2.269199e+04


#### Reorder the columns alphabetically to avoid issues with columns position

In [58]:
if training:
    set_list = ['train','evaluation']
else:
    set_list = ['train','test']

for set_name in set_list:
    Data[set_name].sort_index(axis=1, inplace=True)

#### Save data to save memory

In [59]:
import pickle
import gc
gc.collect()

DATA_LEARNING_FILE = "../data/sales-20180130b"
DATA_EVALUATION_FILE = "../data/evaluation-20180130b"
DATA_TEST_FILE = "../data/test-201801230b"

Data['train'].to_pickle(DATA_LEARNING_FILE)
if training:
    Data['evaluation'].to_pickle(DATA_EVALUATION_FILE)
else:
    Data['test'].to_pickle(DATA_TEST_FILE)

In [ ]:
# 20180127c : with windsorization 0.999 , removing rows for store, item without any sale
# 20180127d : idem without windsorization 
# 20180127e : idem with windsorization 0.9999
# 20180127f : idem with windsorization 0.99
# 20180127g : idem with windsorization 0.99, with full set
# 20180127h : idem with windsorization 0.999, with full set
# 20180128c : windsorization on train only 0.999, with train/eval split
# 20180128d : windsorization on train only 0.99, with train/eval split
# 20180128e : no windsorization, with train/eval split
# 20180128f : with windsor 0.999 on price, with train/eval split
# 20180128g : with windsor 0.999 on price, without eval
# 201801230a : with windsor 0.999 on price, with eval, with lagged item_count, shop_id, item_category_id
# 201801230b : with windsor 0.999 on price, without eval, with lagged item_count, shop_id, item_category_id

### Restart it to retrieve data (optional)

In [38]:
import pickle

training = True

DATA_LEARNING_FILE = "../data/sales-20180130a"
DATA_EVALUATION_FILE = "../data/evaluation-20180130a"

Data = {}

Data['train'] = pd.read_pickle(DATA_LEARNING_FILE)
if training:
    Data['evaluation'] = pd.read_pickle(DATA_EVALUATION_FILE)   

In [42]:
Data['train'].describe()

NameError: name 'Data' is not defined

#### Create train/eval set

In [60]:
clipping  = True

# Random split
#from sklearn.model_selection import train_test_split
#train_set, test_set = train_test_split(Data['train'], test_size = 0.2, random_state = 42)

# Remove the first year to decrease training time (and after we will add delayed values)
Data['train'] = Data['train'][Data['train'].date_block_num >11].sample(frac=1).reset_index(drop=True)

x_train = Data['train'].drop(['item_cnt_month'], axis = 1)

if clipping:
    y_train = Data['train'].item_cnt_month.clip(0,20)
else:
    y_train = Data['train'].item_cnt_month

# I should remove the evaluation prediction rows with missings category

if training:
    x_eval = Data['evaluation'].drop(['item_cnt_month'], axis = 1)
    y_eval = Data['evaluation'].item_cnt_month

del(Data)

In [33]:
from sklearn.metrics import mean_squared_error

def model_evaluation(model_reg, x_train, y_train, x_test, y_test, use_average = True): 
    sales_predictions = model_reg.predict(x_train)
    mse = mean_squared_error(y_train, sales_predictions)
    rmse_train = np.sqrt(mse)

    sales_predictions = pd.DataFrame({'pred': model_reg.predict(x_test.fillna(0))})
    
    if use_average:
        # replace the shop, item rows with no values, with the average on the category 
        missing_shop_item_rows = pd.isnull(x_test.item_shop_mean_price)
        print('Missing lines for shop,items: ', len(x_test[missing_shop_item_rows]))
        sales_predictions.loc[missing_shop_item_rows, 'pred'] = x_test[missing_shop_item_rows].category_cnt_month_mean
    
    # replace the shop, category with no values, with 0 (the shop is not selling this category)
    missing_shop_category_rows = pd.isnull(sales_predictions.pred)  
    print('Missing lines for shop,category: ', len(x_test[missing_shop_category_rows]))
    sales_predictions.loc[missing_shop_category_rows, 'pred'] = 0
    
    mse = mean_squared_error(y_test.clip(0,20), sales_predictions.pred.clip(0,20))
    rmse_test = np.sqrt(mse)

    print("train error: ", '{0:.3f}'.format(rmse_train), "evaluation error: ", '{0:.3f}'.format(rmse_test))

#### Training with Random Forest Regressor

In [61]:
from sklearn.ensemble import RandomForestRegressor

df_reg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=125, n_jobs=5, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
df_reg.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=125, n_jobs=5, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [41]:
model_evaluation(df_reg, x_train, y_train, x_eval, y_eval)

Missing lines for shop,items:  121115
Missing lines for shop,category:  5413
train error:  0.579 evaluation error:  0.968


In [ ]:
# with 20180127c
# max_depth=15, n_estimators=20, max_features: 3 : 1.004 / 1.403
# max_depth=20, n_estimators=20, max_features: 2 : 0.839 / 1.399
# max_depth=20, n_estimators=20, max_features: 3 : 0.818 / 1.394
# max_depth=20, n_estimators=50, max_features: 3 : 0.834 / 1.391 > best
# max_depth=25, n_estimators=20, max_features: 3 : 0.657 / 1.404
# max_depth=20, n_estimators=20, max_features: 4 : 0.795 / 1.410

# with 20180127d, max_depth=20, n_estimators=50, max_features: 3
# train error: 1.104 evaluation error:  4.596 > windsorization is essential!!

# with 20180127e, max_depth=20, n_estimators=25, max_features: 3
# train error:  0.949 evaluation error:  1.612 still not as good

# with 20180127f, max_depth=20, n_estimators=25, max_features: 3
# train error:  0.696 evaluation error:  1.027 > best but beware, 
#I may just be reducing the variance of the evaluation set :)  
# let check with the Kaggle :) 
# with clip 0-20 train error:  0.692 evaluation error:  0.924 (meaningless as it was windsored)

# with 20180127h, max_depth=20, n_estimators=50, max_features: 3, windsor:0.999, full set
# kaggle: 1.41

# with 20180127g, max_depth=20, n_estimators=50, max_features: 3, windsor:0.99, full set
# kaggle: 1.40 > best

# Conclusion: we have brought the results much closer 0.7 / 1.0 / 1.4
# but we are still higher than before ??? 
# does the category do help or harm?

# with 20180127f, removing category stat, max_depth=20, n_estimators=25, max_features: 3, windsor:0.99, full set
# 0.677 evaluation error:  1.043 : slightly worse

# with 20180128d, max_depth=20, n_estimators=25, max_features: 3, windsor on train 0.99
# train error:  0.601 evaluation error:  5.237
# with clipping 0-20: train error:  0.607 evaluation error:  0.979

# with 20180128c, max_depth=20, n_estimators=25, max_features: 3, windsor on train 0.999
# train error:  0.743 evaluation error:  5.039
# with clipping 0-20: train error:  0.743 evaluation error:  0.996

# with 20180128e, max_depth=20, n_estimators=25, max_features: 3, no windsor on train
# train error:  1.095 evaluation error:  4.702
# with clipping 0-20 at evaluation (y_eval and pred): train error:  1.094 evaluation error:  1.002
# with clipping 0-20 of y_train train error:  0.616 evaluation error:  0.975 > best

# with 20180128f, max_depth=20, n_estimators=25, max_features: 3,  windsor 0.999 on price only, clipping 0-20 on train
# train error:  0.616 evaluation error:  0.973 > best

# with 20180128g, max_depth=20, n_estimators=100, max_features: 3,  windsor 0.999 on price only, clipping 0-20 on train
# no eval
# kaggle: 1.02119

# with 20180130a, max_depth=20, n_estimators=25, max_features: 3,  with lagged features, item_id, category_item_id
# 0.567 evaluation error:  0.962 > overfitted...

# with 20180130b, max_depth=20, n_estimators=25, max_features: 3,  with lagged features, without item_id, category_item_id
# 0.579 evaluation error:  0.968 > actually slightly worse
# Kaggle: 

train error:  0.579 evaluation error:  0.968

In [ ]:
from sklearn.externals import joblib
joblib.dump(df_reg, '../models/randomforest_20180128g.pkl')
#df_reg = joblib.load('../models/randomforest_20180127g.pkl') 

In [ ]:
importances = df_reg.feature_importances_
importances

### Xgboost

In [ ]:
import xgboost as xgb

# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBRegressor(max_depth=3, 
                       n_estimators=300, 
                       learning_rate=0.05,
                       n_jobs = 5).fit(x_train, y_train)

if training:
    model_evaluation(gbm, x_train, y_train, x_eval, y_eval, use_average = True)

#### Optimizing the hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [10,50, 100], 'max_features': [2,4,6], 'max_depth': [5, 10, 20]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, n_jobs = 4,
                          scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)

In [ ]:
model_evaluation(grid_search.best_estimator_, x_train, y_train, x_test, y_test)

In [ ]:
grid_search.best_estimator_

#### Training with GBRT

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt_reg = GradientBoostingRegressor(learning_rate = 0.1)
gbrt_reg.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

x_test =  test_set.drop(['item_cnt_month'], axis = 1)
y_test = test_set.item_cnt_month

model_evaluation(gbrt_reg, x_train, y_train, x_test, y_test)

## Result

* 20180120-01 with default randomForestRegressor, only 4 features, eval: 4.90 (not better through grid search)
* idem, but adding the item_category, eval: 5.4 vs 2.42 (train set)
* with GBRT, eval = 7.1 vs 7.36(train set) > not working
* with Random Forest, adding the min, max, mean of sales per store,item eval 4.86 vs 2.04
* idem, removing the item_id: train error:  2.16 test_error:  4.64
* idem, selecting the best estimator: train error:  1.93 test_error:  4.56
* adding the average price per item, per shop and overall, max_features 3, n_estimators=50 train error:  train error:  1.98 test_error:  4.45 (best)
* with one hot encoding of the shop_id: train error:  1.97 test_error:  4.90
* with one hot encoding of the item_category_id: train error:  2.07 test_error:  4.79
* with windsorization of the count_item_day, and the price the results improve a lot: train error:  1.08 test_error:  2.91
* adding the 0 values: train error:  0.33 test_error:  0.94 > kaggle 1.27 without putting missing items to 0 this does not change a thing actually)
* item category is useless > split and provide some price per category
* with new train/test set, not removing the new item prediction: train error:  0.34 test_error:  1.94 / Kaggle: 2.61 (I did not remove the 0 prediction there, nor retrain on the full set)
* with category stats randomforest_20180126b: tain_error:  0.35 test_error:  1.75 Kaggle: 2.4 
* idem, removing missing items: Kaggle: 2.42 it is worse, removing missing category is better!
* with full training: Kaggle: 2.58!!! maybe I am just tragically overfitting
* trying to reduce overfitting by putting max_features = 2/ min_samples_leaf=2: train error:  0.657 test_error:  1.805
* removing the mean, removing the artificial item deletion from validation set: train error:  0.339 evaluation error:  1.768 Kaggle: 

* with removal of missing shop, item rows, and replacement by avrage and 0, 100 estimators: train error:  0.400 evaluation error:  1.417

#### Submission preparation

In [63]:
import pickle

DATA_TEST_FILE = "../data/test-201801230b"

Data = {}

Data['test'] = pd.read_pickle(DATA_TEST_FILE)

Data['test'].head(20)

,ID,category_cnt_month_max,category_cnt_month_mean,date_block_num,item_category_id,item_cnt_month_max,item_cnt_month_mean,item_cnt_month_minus_1,item_cnt_month_minus_12,item_cnt_month_minus_3,item_id,item_overall_mean_price,item_shop_mean_price,month,shop_id,year
0,0,46.0,0.151388,34,19,3.0,0.382353,0.0,1.0,3.0,5037,1960.580473,1693.518519,11,5,2015
1,1,28.0,0.061731,34,55,NaN,NaN,NaN,NaN,NaN,5320,NaN,NaN,11,5,2015
2,2,46.0,0.151388,34,19,3.0,0.294118,1.0,0.0,1.0,5233,844.516003,859.000000,11,5,2015
3,3,26.0,0.119545,34,23,1.0,0.029412,0.0,0.0,1.0,5232,792.527697,599.000000,11,5,2015
4,4,58.0,0.296553,34,20,NaN,NaN,NaN,NaN,NaN,5268,NaN,NaN,11,5,2015
5,5,26.0,0.119545,34,23,3.0,0.323529,1.0,0.0,0.0,5039,1920.578933,1896.305556,11,5,2015
6,6,58.0,0.296553,34,20,3.0,0.147059,2.0,0.0,0.0,5041,3814.782708,3915.666667,11,5,2015
7,7,28.0,0.061731,34,55,5.0,0.235294,0.0,1.0,0.0,5046,326.776255,324.000000,11,5,2015
8,8,28.0,0.061731,34,55,28.0,1.852941,0.0,5.0,2.0,5319,295.510542,299.000000,11,5,2015
9,9,58.0,0.296553,34,20,0.0,0.000000,0.0,0.0,0.0,5003,3977.846154,NaN,11,5,2015


In [64]:
use_average = True

X_test = Data['test'].drop(['ID'], axis = 1)

# Option 1
# It trust the model will learn from the category count for the missing item - 
# for the missing category, I set the prediction to 0
# missing_shop_item_indices = pd.isnull(Data['test']['item_cnt_month_mean'])
# This is not improving the score!

predictions = pd.DataFrame({'pred': df_reg.predict(X_test.fillna(0)).clip(0,20)})

# replace the shop, item rows with no values, with the average on the category

if use_average:
        # replace the shop, item rows with no values, with the average on the category 
        missing_shop_item_rows = pd.isnull(X_test.item_shop_mean_price)
        print('Missing lines for shop,items: ', len(X_test[missing_shop_item_rows]))
        predictions.loc[missing_shop_item_rows, 'pred'] = X_test[missing_shop_item_rows].category_cnt_month_mean

# for the shop with not category, replace with 

missing_shop_category_rows = pd.isnull(predictions.pred)
print('Missing lines for shop,category: ', len(X_test[missing_shop_category_rows]))
predictions.loc[missing_shop_category_rows, 'pred'] = 0

# Create the submission file:

submission = data_load['sample_submission'].copy()

submission.loc[:, 'item_cnt_month'] = predictions.pred
submission.head(10)

Missing lines for shop,items:  102796
Missing lines for shop,category:  0


,ID,item_cnt_month
0,0,0.455219
1,1,0.061731
2,2,0.953177
3,3,0.044127
4,4,0.296553
5,5,0.532677
6,6,0.768199
7,7,0.172027
8,8,0.629908
9,9,0.296553


In [65]:
SUBMISSION_FILE = "../data/sales_sub_20180130b.csv"

submission.to_csv(SUBMISSION_FILE, index = False)

### One-hot encoding (optional, only for DNN)

In [ ]:
# One hot encoding of shop_id
# The test set has only a few shops, so we have to use scikitlearn onehotencoder

cols = ['shop_id']

enc = preprocessing.OneHotEncoder()

# FIT
enc.fit(Data['train'][cols])

# Transform
for set_name in ['train', 'test']:
    vec_data= pd.DataFrame(enc.transform(Data[set_name][cols]).toarray())
    vec_data.columns = ["shop_id_" + str(i) for i in range(enc.feature_indices_[1])]
    vec_data.index = Data[set_name].index
    Data[set_name] = Data[set_name].drop(cols, axis=1)
    Data[set_name] = Data[set_name].join(vec_data)
    
# One hot encoding of item_category_id
# The test set has only a few shops, so we have to use scikitlearn onehotencoder

cols = ['item_category_id']

enc = preprocessing.OneHotEncoder()

# FIT
enc.fit(Data['train'][cols])

# Transform
for set_name in ['train', 'test']:
    vec_data= pd.DataFrame(enc.transform(Data[set_name][cols]).toarray())
    vec_data.columns = ["item_category_id_" + str(i) for i in range(enc.feature_indices_[1])]
    vec_data.index = Data[set_name].index
    Data[set_name] = Data[set_name].drop(cols, axis=1)
    Data[set_name] = Data[set_name].join(vec_data)   

In [ ]:
y_train.describe()

In [ ]:
from sklearn.metrics import mean_squared_error


sales_predictions = pd.DataFrame(Data['evaluation'].item_cnt_month.copy())
sales_predictions.item_cnt_month = 0.28



mse = mean_squared_error(Data['evaluation'].item_cnt_month, sales_predictions.item_cnt_month)
rmse_test = np.sqrt(mse)
rmse_test

In [ ]:
y_train.describe()

In [ ]:
Data['train']['item_cnt_month_minus_12'] = 0
Data['train']['item_cnt_month_minus_3'] = 0
Data['train']['item_cnt_month_minus_1'] = 0 

for month in range(12, 33,1):
    condition = Data['train'].date_block_num == month
    Data['train'].loc[condition, 'item_cnt_month_minus_12'] = Data['train'][Data['train'].date_block_num == (month-12)]
    Data['train'].loc[condition, 'item_cnt_month_minus_3'] = Data['train'][Data['train'].date_block_num == (month-3)]
    Data['train'].loc[condition, 'item_cnt_month_minus_1'] = Data['train'][Data['train'].date_block_num == (month-1)]

if training:
    Data['evaluation']['item_cnt_month_minus_12'] = 0
    Data['evaluation']['item_cnt_month_minus_3'] = 0
    Data['evaluation']['item_cnt_month_minus_1'] = 0 

    Data['evaluation'].loc[:, 'item_cnt_month_minus_12'] = Data['train'][Data['train'].date_block_num == (33-12)]
    Data['evaluation'].loc[:, 'item_cnt_month_minus_3'] = Data['train'][Data['train'].date_block_num == (33-3)]
    Data['evaluation'].loc[:, 'item_cnt_month_minus_1'] = Data['train'][Data['train'].date_block_num == (33-1)]    

# Merge version for evaluation and test
Data['test']['item_cnt_month_minus_12'] = 0
Data['test']['item_cnt_month_minus_3'] = 0
Data['test']['item_cnt_month_minus_1'] = 0 